AutoGluon is an auto-ml package, developed by J Mueller, X Shi, A Smola:

Mueller, Jonas, Xingjian Shi, and Alexander Smola. "Faster, Simpler, More Accurate: Practical Automated Machine Learning with Tabular, Text, and Image Data." Proceedings of the 26th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining. 2020.

For tabular data, AutoGluon can produce models to predict the values in one column based on the values in the other columns. With just a single call to fit(), you can achieve high accuracy in standard supervised learning tasks (both classification and regression).

In the economy of a competition it can help you to create benchmarks, get insights on models' workings and accelerate your experimentations.

Installing the latest Scikit-learn

In [ ]:
!pip install scikit-learn -U

Installing LightGBM for GPU

In [ ]:
!pip install --upgrade --force-reinstall --install-option=--gpu lightgbm

Installing AutoGluon

In [ ]:
!pip install autogluon 

In [ ]:
# Importing core libraries
import numpy as np
import pandas as pd
import gc

# Importing AutoGluon
from autogluon.tabular import TabularDataset, TabularPredictor

# Scikit Learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.pipeline import Pipeline

In [ ]:
# Derived from the original script https://www.kaggle.com/gemartin/load-data-reduce-memory-usage 
# by Guillaume Martin

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# Loading data 
X_train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv").set_index('id')
X_test = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv").set_index('id')

In [ ]:
# Feature engineering
unique_values = X_train.iloc[:1000].nunique()
categoricals = [col for col in  unique_values.index[unique_values < 10] if col!='target']
numeric = [col for col in X_test.columns if col not in categoricals]

X_train['mean_numeric'] = X_train[numeric].mean(axis=1)
X_train['std_numeric'] = X_train[numeric].std(axis=1)
X_train['min_numeric'] = X_train[numeric].min(axis=1)
X_train['max_numeric'] = X_train[numeric].max(axis=1)
X_train['sum_categoricals'] = X_train[categoricals].sum(axis=1)

X_test['mean_numeric'] = X_test[numeric].mean(axis=1)
X_test['std_numeric'] = X_test[numeric].std(axis=1)
X_test['min_numeric'] = X_test[numeric].min(axis=1)
X_test['max_numeric'] = X_test[numeric].max(axis=1)
X_test['sum_categoricals'] = X_test[categoricals].sum(axis=1)

In [ ]:
# Feature selection
features = ['f1', 'f103', 'f107', 'f112', 'f119', 'f12', 'f125', 'f127', 'f13', 
            'f130', 'f134', 'f136', 'f138', 'f139', 'f14', 'f141', 'f143', 'f144', 
            'f150', 'f152', 'f154', 'f156', 'f16', 'f163', 'f169', 'f17', 'f173', 
            'f179', 'f18', 'f187', 'f19', 'f192', 'f195', 'f198', 'f2', 'f20', 'f200', 
            'f201', 'f211', 'f213', 'f214', 'f22', 'f222', 'f227', 'f231', 'f239', 'f241', 
            'f243', 'f247', 'f252', 'f258', 'f26', 'f266', 'f27', 'f29', 'f3', 'f33', 'f4', 
            'f40', 'f42', 'f43', 'f44', 'f48', 'f5', 'f52', 'f53', 'f56', 'f58', 'f6', 'f60', 
            'f62', 'f63', 'f64', 'f65', 'f69', 'f7', 'f71', 'f72', 'f73', 'f74', 'f75', 'f77', 
            'f78', 'f8', 'f82', 'f83', 'f85', 'f86', 'f90', 'f92', 'f93', 'f95', 'f96', 'f98', 
            'f99', 'mean_numeric', 'std_numeric', 'sum_categoricals']

X_train = X_train[features + ['target']]
X_test = X_test[features]

In [ ]:
### REDUCE MEMORY USAGE
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)
gc.collect()

In [ ]:
VALIDATION = False
if VALIDATION is True:
    X_train, X_val = train_test_split(X_train, test_size=int(len(X_train) * 0.2), random_state=42)
    train_data = TabularDataset(X_train)
    val_data = TabularDataset(X_val)
else:
    train_data = TabularDataset(X_train)
    val_data = TabularDataset(X_train.iloc[:100_000, :])

SUBSAMPLE = False
RANDOM_STATE = 0
if SUBSAMPLE is True:
    subsample_size = 100_000  # subsample subset of data for faster demo, try setting this to much larger values
    train_data = train_data.sample(n=subsample_size, random_state=RANDOM_STATE)
    
train_data.head()

In [ ]:
label = 'target'
print("Summary of target variable: \n", train_data[label].describe())

In [ ]:
!mkdir agModels

You can actually use the optimized parameters that you can find on public kernels to boost your AutoGluon performances.

For instance these parameters are from the high scoring notebook https://www.kaggle.com/dlaststark/tps-1021-la-dee-da by DLASTSTARK

In [ ]:
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'use_label_encoder': False,
    'n_estimators': 10000,
    'max_depth': 3,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'learning_rate': 0.01187,
    'gpu_id': 0,
    'predictor': 'gpu_predictor'
}

cb_params = {
    'loss_function' : 'CrossEntropy',
    'eval_metric' : 'AUC',
    'iterations' : 10000,
    'grow_policy' : 'SymmetricTree',
    'use_best_model' : True,
    'depth' : 5,
    'l2_leaf_reg' : 3.0,
    'random_strength' : 1.0,
    'learning_rate' : 0.1,
    'task_type' : 'GPU',
    'devices' : '0',
    'verbose' : 0
}

lgb_params = {
    'objective' : 'binary',
    'metric' : 'auc',
    'max_depth' : 3,
    'num_leaves' : 7,
    'n_estimators' : 5000,
    'colsample_bytree' : 0.3,
    'subsample' : 0.5,
    'reg_alpha' : 18,
    'reg_lambda' : 17,
    'learning_rate' : 0.095,
    'device' : 'gpu'
}

In [ ]:
save_path = 'agModels'  # specifies folder to store trained models
presets='best_quality'
metric = 'roc_auc'
hours = 5.0
predictor = (TabularPredictor(label=label, eval_metric=metric, path=save_path)
             .fit(train_data,
                  excluded_model_types = ['KNN', 'XT' ,'RF', 'NN', 'FASTAI'],
                  hyperparameters = {'GBM': lgb_params, 
                                     'CAT': cb_params,
                                     'XGB': xgb_params
                                    },
                  presets=presets,
                  time_limit= int(60 * 60 * hours))
            )


In [ ]:
results = predictor.fit_summary(show_plot=True)

In [ ]:
leaderboard = predictor.leaderboard(val_data)

In [ ]:
test_data = TabularDataset(X_test)
test_preds = predictor.predict_proba(test_data)

In [ ]:
# Predicting and submission
submission = pd.DataFrame({'id':X_test.index, 
                           'target': test_preds.iloc[:,1].ravel()})

submission.to_csv("./submission.csv", index=False)
submission.to_csv("./agModels/submission.csv", index=False)

In [ ]:
submission